<a href="https://colab.research.google.com/github/kovzanok/dls-final-task/blob/main/2_ArcFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ArcFace Loss (Additive Angular Margin Loss)

## Теория ArcFace

В случае с обучением на задачу классификации первая подходящая лосс-функция, которая нам приходит в голову — Cross-Entropy. И на ней действительно можно обучать сеть для распознавания лиц. Но за много лет люди придумали более хитрые лосс-функции, которые делают обучение сети для распознавания лиц более эффективным. Одним из лучших считается ArcFace Loss (Additive Angular Margin Loss).

Этот лосс — чуть измененная кросс-энтропия. Он позволяет достичь лучшего распределения векторов лиц на сфере. В нем добавлены некоторые дополнительные ограничения и гиперпараметры, для того чтобы эмбеддинги лиц одного класса были более близки между собой, а эмбеддинги лиц разных людей оставались далеки. То есть, этот лосс позволяет лучше кластеризовать лица на сфере единичного радиуса.


**Как устроен ArcFace**:

Стандартные SoftMax + кросс-энтропия (CE) выглядят так:

$$L_{CE} = \frac{-1}{N}\sum_1^N \frac{e^{W_{y_i}^{T}x_i + b_{y_i}}}{\sum^n_{j=1}e^{W_j^Tx_i+b_j}},$$

здесь:
- $x_i \in \mathbb{R^d}$ — вектор $i$-го элемента обучающей выборки перед последним полносвязным слоем сети. $y_i$ — класс этого элемента;
- $W_j \in \mathbb{R^d}$ — j-ый столбец матрицы весов последнего слоя сети (т.е. слоя, который производит итоговую классификацю входящего объекта);
- $b_j \in \mathbb{R^d}$ — j-ый элемент вектора байеса последнего слоя сети;
- $N$ — batch size;
- $n$ — количество классов.


Хотя этот лосс работает хорошо, он явным образом не заставляет эмбеддинги $x_i$ элементов, принадлежащих одному классу, быть близкими друг к другу по расстоянию. И не заставляет эмбеддинги элементов, принадлежащих разным классам, быть далеко друг от друга. Все, что хочет этот лосс — чтобы на основе эмбеддингов $x_i$ можно было хорошо классифицировать элементы, никакие ограничений на расстояния между эмбеддингами $x_i$ он не вводит.

Из-за этого у нейросетей для распознавания лиц, которые обучены на обычном CE loss, бывают проблемы с распознаванием лиц, которые сильно отличаются от лиц того же человека разными допатрибутами (шляпа/прическа/очки и т.п.). Просто эмбеддинг для таких лиц получается довольно далек по расстоянию от других эмбеддингов лиц этого же человека.

Давайте теперь немного поправим формулу:
- уберем байес последнего слоя, т.е. сделаем $b_j=0$;
- нормализуем веса последнего слоя: ||$W_j$|| = 1;
- нормализуем эмбеддинги: ||$x_i$|| = 1. Перед подачей их на вход последнему слою (т.е. перед умножением на матрицу $W_j$) умножим их на гиперпараметр s. По сути, мы приводим норму всех эмбеддингов к s. Смысл этого гиперпараметра в том, что, возможно, сети проще будет классифицировать эмбеддинги, у которых не единичная норма.

Нормализация эмбеддингов приводит к тому, что эмбеддинги начинают быть распределены по сфере единичного радиуса (и сфере радиуса s после умножения на ниперпараметр s). И итоговые предсказания сети после последнего слоя зависят только от угла между эмбеддингами $x_i$ и выученных весов $W_j$. От нормы эмбеддинга $x_i$ они больше не зависят, т.к. у всех эмбеддингов они теперь одинаковые.

Получается, в степени экспоненты у нас останется выражение $s W_{y_i}^{T}x_i$, которое можно переписать в виде  $s W_{y_i}^{T}x_i = s ||W_{y_i}||\cdot ||x_i|| \cdot cos\Theta_{y_i}$. Тут $\Theta_{y_i}$ — это угод между векторами $W_{y_i}$ и $x_i$. Но так как мы сделали нормы $W_{y_i}$ и $x_i$ единичными, то все это выражение просто будет равно $s cos\Theta_{y_i}$.

В итоге мы получим следующую формулу лосса:

$$L = \frac{-1}{N}\sum_1^N \frac{e^{s\ cos\Theta_{y_i}}}{e^{s\ cos\Theta_{y_i}} + \sum^n_{j=1,\ j\ne y_i} e^{s\ cos\Theta_j}}$$


И последний шаг. Добавим еще один гиперпараметр $m$. Он называется additive angular margin penalty и заставляет эмбеддинги одного класса быть ближе друг к другу, а эмбеддинги разных классов — более далекими друг от друга.

В итоге получим вот что:

$$L_{ArcFace} = \frac{-1}{N}\sum_1^N \frac{e^{s\ cos(\Theta_{y_i} + m)}}{e^{s\ cos(\Theta_{y_i} + m)} + \sum^n_{j=1,\ j\ne y_i} e^{s\ cos\Theta_j}}$$

Это и есть ArcFace Loss с двумя  гиперпараметрами, s и m.

Получается, что ArcFace Loss завтавляет сеть выучивать эмбеддинги, распределенные по сфере радиуса s, причем чтобы эмбеддинги одного класса были ближе друг к другу, а эмбеддинги разных классов — более далеки друг от друга.



**Доплитература по ArcFace Loss:**

Оригинальная статья: https://arxiv.org/pdf/1801.07698.pdf

## Другие лоссы

Кроме ArcFace, есть еще много разных вариантов лоссов для задачи face recognition. Некоторые из них можно найти, например, [тут](https://openaccess.thecvf.com/content_CVPRW_2020/papers/w48/Hsu_A_Comprehensive_Study_on_Loss_Functions_for_Cross-Factor_Face_Recognition_CVPRW_2020_paper.pdf). Вы можете попробовать реализовать другие лосс-функции в этом проекте в качестве дополнительного задания.

Кроме этого, можно миксовать лосс-функции. Например, обучать нейросеть на сумме ArcFace и TripletLoss. Часто так выходит лучше, чем если обучать на каком-то одном лоссе.

# Датасет

В качестве датасета нужно использовать картинки из CelebA, выровненные при помощи своей модели из задания 1. Очень желательно их еще кропнуть таким образом, чтобы нейросети поступали на вход преимущественно только лица без какого либо фона, частей тела и прочего. Целиком брать весь датасет CelebA не обязательно, он слишком большой.

Если планируете делать дополнительное задание на Identificaton rate metric, то **обязательно разбейте заранее датасет на train/val или train/val/test.** Это нужно сделать не только на уровне кода, а на уровне папок, чтобы точно знать, на каких картинках модель обучалась, а на каких нет. Лучше заранее почитайте [ноутбук с заданием](https://colab.research.google.com/drive/1sjO2-N8EsLb2HQcOELikFQCqKUWrPehJ?usp=sharing).

# План заданий

Итак, вот, что от вас требуется в этом задании:

* Выбрать модель (или несколько моделей) для обучения. Можно брать предобученные на ImageNet, но нельзя использовать модели, предобученные на задачу распознавания лиц.
* Обучить эту модель (модели) на CE loss. Добиться accuracy > 0.7.
* Реализовать ArcFace loss.
* Обучить модель (модели) на ArcFace loss. Добиться accuracy > 0.7.
* Написать небольшой отчет по обучению, сравнить CE loss и ArcFace loss.

**P.S. Не забывайте сохранять модели после обучения**

#Зависимости и загрузка данных

In [1]:
!gdown 1SLMKar0E9fgOvhrzqp7kwpk27MnSA39s

Downloading...
From (original): https://drive.google.com/uc?id=1SLMKar0E9fgOvhrzqp7kwpk27MnSA39s
From (redirected): https://drive.google.com/uc?id=1SLMKar0E9fgOvhrzqp7kwpk27MnSA39s&confirm=t&uuid=3a26fc6f-13e2-4f94-8621-2b04f1593ebc
To: /kaggle/working/best_recognition_model.pth
100%|██████████████████████████████████████| 61.6M/61.6M [00:00<00:00, 62.2MB/s]


In [2]:
from torch import nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from PIL import Image
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torchvision.models as models
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import torchvision.transforms.functional as TF
import cv2
from sklearn.preprocessing import LabelEncoder


import random
import os

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [4]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

#Датасет

In [5]:
path = '/kaggle/input/stage-3-dataset/identity_CelebA.txt'

df = pd.read_csv(path, sep='\s+', header=None,index_col=0)

In [6]:
df.index.name = 'image_name'
df.rename(columns={1:'id'},inplace=True)

In [7]:
df.head()

,id
image_name,
000001.jpg,2880
000002.jpg,2937
000003.jpg,8692
000004.jpg,5805
000005.jpg,9295


In [8]:
def get_filenames_os(folder_path):
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    return files

In [11]:
path = '/kaggle/input/stage-3-dataset/file/content/stage_3_dataset'

filenames = get_filenames_os(path)

In [12]:
filtered_df = df.loc[filenames]


In [13]:
le = LabelEncoder()
filtered_df['encoded_id'] = le.fit_transform(filtered_df['id'])

In [14]:
class CelebADataset(Dataset):
    def __init__(self, img_dir, img_names, image_df, transform=None):
        super().__init__()
        self.img_dir = img_dir
        self.transform = transform
        self.image_names = img_names
        self.image_df = image_df

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        id = self.image_df.loc[img_name]['encoded_id']

        return image, int(id)


In [15]:
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomAffine(degrees=5, translate=(0.02, 0.02)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3),
])


path = '/kaggle/input/stage-3-dataset/file/content/stage_3_dataset'

dataset = CelebADataset(
    img_dir=path,
    transform=transform,
    image_df=filtered_df,
    img_names = filtered_df.index.to_list()
)

In [16]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [17]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2,pin_memory=True)
val_loader = DataLoader(test_dataset, batch_size=128, num_workers=2,pin_memory=True)

#Модель

In [18]:
num_classes = int(filtered_df.nunique().values[0])

model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, num_classes)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 148MB/s] 


In [19]:
path = '/kaggle/working/best_recognition_model.pth'

model.load_state_dict(torch.load(path))

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [20]:
for name, param in model.named_parameters():
    if 'layer4' in name or 'fc' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False


In [21]:
model = model.to(device)

#Обучение

In [22]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.05)
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

In [23]:
def train_epoch(model, criterion, optimizer, train_loader):
    model.train()
    running_loss = 0
    for image, label in tqdm(train_loader,leave=False):
        image, label = image.to(device), label.to(device)

        logit = model(image)
        loss = criterion(logit, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)

    return avg_loss



In [24]:
@torch.no_grad()
def val_epoch(model, criterion,val_loader):
    model.eval()

    running_loss = 0
    correct = 0
    total = 0

    for image, label in tqdm(val_loader, leave=False):
        image, label = image.to(device), label.to(device)
        logit = model(image)

        loss = criterion(logit, label)

        running_loss += loss.item()
        _, predicted = torch.max(logit, 1)

        correct += (predicted == label).sum().item()
        total += label.size(0)

    avg_loss = running_loss / len(val_loader)
    accuracy = correct / total

    return avg_loss, accuracy

In [25]:
path = '/kaggle/working/best_recognition_model_kaggle.pth'

def train(model, criterion, optimizer, train_loader, val_loader, num_epochs=100, scheduler=None):
    history = {'train':[],'test':[],'accuracy':[]}
    best_val_loss = float('inf')
    best_accuracy = 0

    for epoch in tqdm(range(num_epochs)):
        train_loss = train_epoch(model, criterion, optimizer, train_loader)
        history['train'].append(train_loss)

        val_loss, accuracy = val_epoch(model, criterion, val_loader)
        history['test'].append(val_loss)
        if scheduler:
          scheduler.step(val_loss)
        print(f"Epoch {epoch + 1} — Train Loss: {train_loss:.4f} — Val Loss: {val_loss:.4f} - Accuracy on val: {accuracy}")

        # if accuracy > best_accuracy:
        #     best_accuracy = accuracy
        #     torch.save(model.state_dict(), path)
        #     print("✅ Model saved (best so far)")


    return history

In [26]:
history = train(model, criterion, optimizer, train_loader, val_loader, scheduler=scheduler)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/950 [00:00<?, ?it/s]

KeyboardInterrupt: 